In [ ]:
from pathlib import Path

import tensorflow as tf
import PIL
import PIL.Image

data_dir = Path("data/cards/kind")
creatures = data_dir.glob("creature/*.jpg")
PIL.Image.open(str(next(creatures)))

: 